# Prepare CNN Data

## Setup

In [1]:
# USE ACCOUNT: robmarty3@gmail.com
import ee
#ee.Authenticate()
ee.Initialize()

In [2]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time
from datetime import datetime

#import geetools
#from geetools import ui, cloud_mask
#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

## Parameters

In [3]:
#224
SURVEY_NAME = 'DHS'
SATELLITE = 'landsat_7' # 's2'; 'landsat'; 'landsat_7'
OUTCOME_VAR = "ntlharmon" # "ntlharmon" OR "viirs"
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 1 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

if SATELLITE == 's2':
    KERNEL_SIZE = 224
elif SATELLITE == 'landsat':
    KERNEL_SIZE = 224 #167
elif SATELLITE == 'landsat_7':
    KERNEL_SIZE = 224 #167

print(KERNEL_SIZE)

224


In [4]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

out_path_errors = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE)

## Load Data

In [5]:
# Relies on VIIRs Values
if OUTCOME_VAR == 'viirs':
    survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'data_for_cnn.csv'))
elif OUTCOME_VAR == 'ntlharmon':
    survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'data_for_cnn_ntlharmon.csv'))
    
print(survey_df.shape)
print(survey_df.ntl_group.value_counts())

(128177, 8)
1    57833
0    41292
2    29052
Name: ntl_group, dtype: int64


In [6]:
# Remove observations that had issues
if SATELLITE == 'landsat_7':
    survey_df = survey_df[survey_df['uid'] != 'BO200800002050']
    survey_df = survey_df[survey_df['uid'] != 'CO201000000592']

if SATELLITE == 's2':
    #survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]
    survey_df = survey_df[survey_df['uid'] != 'CO201000003682']
    survey_df = survey_df[survey_df['uid'] != 'CO201000003683']
    survey_df = survey_df[survey_df['uid'] != 'CO201000004885']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180058']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180040']
    survey_df = survey_df[survey_df['uid'] != 'IA201400300032']
    survey_df = survey_df[survey_df['uid'] != 'IA201400010147']
    survey_df = survey_df[survey_df['uid'] != 'NM201300000001']
    survey_df = survey_df[survey_df['uid'] != 'CO201000002924']
    survey_df = survey_df[survey_df['uid'] != 'CO201000002524']
    survey_df = survey_df[survey_df['uid'] != 'CO201000001170']
    survey_df = survey_df[survey_df['uid'] != 'CO201000003554']
    survey_df = survey_df[survey_df['uid'] != 'GA201200000310']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180104'] 
    survey_df = survey_df[survey_df['uid'] != 'IA201400180012']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180011']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180048']
    survey_df = survey_df[survey_df['uid'] != 'IA201400300002']
    survey_df = survey_df[survey_df['uid'] != 'IA201400130192']
    survey_df = survey_df[survey_df['uid'] != 'IA201400140845']
    survey_df = survey_df[survey_df['uid'] != 'IA201400010076']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180133']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180123']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180047']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180138']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180052']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180055']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180116']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180030']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180086']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180050']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180027']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180072']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180080']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180079']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180064']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180081']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180062']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000335']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000330']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000322']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180062']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180140']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180100']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000342']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180081']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180028']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000340']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180091']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180112']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000331']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000336']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000334']
    survey_df = survey_df[survey_df['uid'] != 'IA201400180083']
    survey_df = survey_df[survey_df['uid'] != 'GY200900000293']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000328']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000327']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000333']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000332']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000329']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000339']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000326']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000323']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000321']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000338']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000324']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000325']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000337']
    survey_df = survey_df[survey_df['uid'] != 'UG201800000341']

In [7]:
# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

len(tf_record_list)

1104

In [8]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]
    
print(len(tf_record_list))

913


In [ ]:
## Blank error dataframe
errors_df = pd.DataFrame()

## Error file name
now = datetime.now()
current_time = now.strftime("%d_%m_%y_%H_%M_%S")
error_file_name = 'errors_' + current_time + '.csv'

if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.

        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            
            try:
            
                time.sleep(3)
                print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                proto_examples_all.extend(proto_examples_i)
                
            except:
                
                print("Error ---")
                print(survey_df_yeari_chunki['uid'])
                
                errors_df = errors_df.append(survey_df_yeari_chunki[['uid']], ignore_index = True)
                errors_df.to_csv(os.path.join(out_path_errors, error_file_name))
                                 
                time.sleep(15)
                pass

        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")



/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_80651/2073147091.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Putting 250 observations into forcnn_test_IA_2_2_all.tfrecord
Observation: 0/250


2022-06-02 15:46:40.606300: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Observation: 1/250
Observation: 2/250
Observation: 3/250
Observation: 4/250
Observation: 5/250
Observation: 6/250
Observation: 7/250
Observation: 8/250
Observation: 9/250
Observation: 10/250
Observation: 11/250
Observation: 12/250
Observation: 13/250
Observation: 14/250
Observation: 15/250
Observation: 16/250
Observation: 17/250
Observation: 18/250
Observation: 19/250
Observation: 20/250
Observation: 21/250
Observation: 22/250
Observation: 23/250
Observation: 24/250
Observation: 25/250
Observation: 26/250
Observation: 27/250
Observation: 28/250
Observation: 29/250
Observation: 30/250
Observation: 31/250
Observation: 32/250
Observation: 33/250
Observation: 34/250
Observation: 35/250
Observation: 36/250
Observation: 37/250
Observation: 38/250
Observation: 39/250
Observation: 40/250
Observation: 41/250
Observation: 42/250
Observation: 43/250
Observation: 44/250
Observation: 45/250
Observation: 46/250
Observation: 47/250
Observation: 48/250
Observation: 49/250
Observation: 50/250
Observati

/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_80651/2073147091.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors_df = errors_df.append(survey_df_yeari_chunki[['uid']], ignore_index = True)


Observation: 111/250
Observation: 112/250
Observation: 113/250
Observation: 114/250
Observation: 115/250
Observation: 116/250
Observation: 117/250
Observation: 118/250
Observation: 119/250
Observation: 120/250
Observation: 121/250
Observation: 122/250
Observation: 123/250
Observation: 124/250
Observation: 125/250
Observation: 126/250
Observation: 127/250
Observation: 128/250
Observation: 129/250
Observation: 130/250
Observation: 131/250
Observation: 132/250
Observation: 133/250
Observation: 134/250
Observation: 135/250
Observation: 136/250
Observation: 137/250
Observation: 138/250
Observation: 139/250
Observation: 140/250


In [9]:
if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.
        try:

            survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
            year_i = survey_df_yeari['year'].iloc[0]

            ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
            survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

            print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

            proto_examples_all = []
            for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
                time.sleep(3)
                print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                proto_examples_all.extend(proto_examples_i)

            ### Save data as tf record
            out_path_i = os.path.join(out_path, tfr_i)
            print(out_path_i)
            with tf.io.TFRecordWriter(out_path_i) as writer:
                for tf_example in proto_examples_all:
                    writer.write(tf_example.SerializeToString())

            print("Success \o/")

        except:
            print("Error ---")
            print(survey_df_yeari_chunki['uid'])
            time.sleep(15)
            pass

/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_18156/3807929331.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Putting 42 observations into forcnn_test_BO_1_1_all.tfrecord
Observation: 0/42


2022-06-02 11:27:51.815818: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Observation: 1/42
Observation: 2/42
Observation: 3/42
Observation: 4/42
Error ---
4739    BO200800002050
Name: uid, dtype: object


/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_18156/3807929331.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


...

/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_18156/3807929331.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Putting 145 observations into forcnn_test_IA_5_3_all.tfrecord
Observation: 0/145
Error ---
2129    IA201400010160
Name: uid, dtype: object


/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_18156/3807929331.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Putting 76 observations into forcnn_test_MW_4_1_all.tfrecord
Observation: 0/76
Observation: 1/76
Observation: 2/76
Observation: 3/76
Observation: 4/76
Observation: 5/76
Observation: 6/76
Observation: 7/76
Observation: 8/76
Observation: 9/76
Observation: 10/76
Observation: 11/76
Observation: 12/76
Observation: 13/76
Observation: 14/76
Observation: 15/76
Observation: 16/76
Observation: 17/76
Observation: 18/76
Observation: 19/76
Observation: 20/76
Observation: 21/76
Observation: 22/76
Observation: 23/76
Observation: 24/76
Observation: 25/76
Observation: 26/76
Observation: 27/76
Observation: 28/76
Observation: 29/76
Observation: 30/76
Observation: 31/76
Observation: 32/76
Observation: 33/76
Error ---
2705    MW200000001030
Name: uid, dtype: object


KeyboardInterrupt: 

## Troubleshooting

In [ ]:
survey_df = survey_df_yeari_chunki
satellite_name = SATELLITE
kernel_size = KERNEL_SIZE
year = year_i

In [11]:
import ee
import numpy as np
import geetools
from geetools import ui, cloud_mask
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf

cloud_mask_landsatSR = cloud_mask.landsatSR()
cloud_mask_sentinel2 = cloud_mask.sentinel2()

# tfrecord helper functions ----------------------------------------------------
# https://stackoverflow.com/questions/52324515/passing-multiple-inputs-to-keras-model-from-tf-dataset-api
# https://www.tensorflow.org/tutorials/load_data/tfrecord

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def chunk_ids(total_length, chunk_size):
    n_numbers = np.ceil(total_length / chunk_size)
    n_numbers = int(n_numbers)
    
    chunk_ids = list(range(0,n_numbers)) * chunk_size
    chunk_ids.sort()
    chunk_ids = chunk_ids[:total_length]
    
    return chunk_ids

# Main Functions -----------------------------------------------------------------
def survey_to_fc(survey_df):
    '''
    Convert pandas dataframe of survey locations to a feature collection. 
    
    Inputs:
        survey_df: pandas dataframe of survey locations. Function assumes 
                   the dataframe contains (1) latitude, (2) longitude and
                   (3) uid variables. Assumes coordinates in WGS84.
    Returns:
        (feature collection)
    '''
    
    survey_fc_list = []
    
    n_rows = survey_df.shape[0]
    for i in range(0, n_rows):
        survey_df_i = survey_df.iloc[[i]]

        f_i = ee.Feature(ee.Geometry.Point([survey_df_i['longitude'].iloc[0], 
                                            survey_df_i['latitude'].iloc[0]]), 
                         {'uid': survey_df_i['uid'].iloc[0]})

        survey_fc_list.append(f_i)
        
    survey_fc = ee.FeatureCollection(survey_fc_list)
    
    return survey_fc

def normalized_diff(values1, values2):
    '''
    Normalized Difference Value

    Input:  values1, values2 (must be same dimensions)

    Output: np array
    '''

    return (values2 - values1)/(values2 + values1)

def ee_to_np_daytime(daytime_f, survey_df, n_rows, b_b, g_b, r_b): # nir_b, swir_b
    '''
    Transforms feature collection from neighborhood array to np array. Stacks bands
    so that they are: NTL, blue, green, red, NDVI, other single daytime bands

    Input:  
      f (features)
      n_rows (number of features)

    Output: np array
    '''
    
    example_proto_list = []

    for i in range(0, n_rows):
        survey_uid = survey_df['uid'].iloc[i]
        #folder_name = survey_df['tf_folder_name'].iloc[i]
        viirs_ntl_group = int(survey_df['ntl_group'].iloc[i])
        survey_year_i = int(survey_df['year'].iloc[i])
        uid_i = survey_df['uid'].iloc[i].encode()
        
        d_f_i = daytime_f[i]['properties']
        #n_f_i = ntl_f[i]['properties']

        # SAVE AS TFRECORD

        # Prep Files
        ### RGB
        brgb_l = [np.array(d_f_i[r_b]), np.array(d_f_i[g_b]), np.array(d_f_i[b_b])]
        brgb_np = np.stack(brgb_l, axis=-1)
        brgb_np = brgb_np.astype(np.uint16)
        brgb_np_tf = tf.io.encode_png(brgb_np, compression = 9)
        #brgb_np_tf = tf.io.serialize_tensor(brgb_np)
        
        ### NIR
        if False:
            bnir_np = d_f_i[nir_b]      
            bnir_np = np.expand_dims(bnir_np, axis=2) # original (224, 224), change to (224,224,1) -> so can stack
            bnir_np = bnir_np.astype(np.uint16)
            bnir_np_tf = tf.io.encode_png(bnir_np, compression = 9)
            #bndvi_np_tf = tf.io.serialize_tensor(bndvi_np)

        if True:
            # https://www.tensorflow.org/api_docs/python/tf/io/encode_png
            ### NDVI 
            bndvi_np = d_f_i['NDVI']      
            bndvi_np = np.expand_dims(bndvi_np, axis=2) # original (224, 224), change to (224,224,1) -> so can stack
            # Convert from -1 to 1 to 0 to 20000
            bndvi_np = bndvi_np + 1
            bndvi_np = bndvi_np * 10000
            bndvi_np = bndvi_np.astype(np.uint16)
            bndvi_np_tf = tf.io.encode_png(bndvi_np, compression = 9)
            #bndvi_np_tf = tf.io.serialize_tensor(bndvi_np)

            ### BU 
            bbu_np = d_f_i['BU']      
            bbu_np = np.expand_dims(bbu_np, axis=2) # original (224, 224), change to (224,224,1) -> so can stack
            # Convert from -1 to 1 to 0 to 20000
            bbu_np = bbu_np + 1
            bbu_np = bbu_np * 10000
            bbu_np = bbu_np.astype(np.uint16)
            bbu_np_tf = tf.io.encode_png(bbu_np, compression = 9)
            #bndvi_np_tf = tf.io.serialize_tensor(bndvi_np)

        ### NTL
        # Not uint16, so so serialize
        #bntl_np = np.array(n_f_i['avg_rad'])
        #bntl_np = np.expand_dims(bntl_np, axis=2)
        # Values to uint16
        #bntl_np = bntl_np + 2 # Can be negative
        #bntl_np = bntl_np * 100 # consider two decimal places before uint16 // could also to * 10 (second decimal may not matter)
        #bntl_np[bntl_np >= 65535] = 65535 # within range of uint16
        #bntl_np = bntl_np.astype(np.uint16)
        #bntl_np_tf = tf.io.encode_png(bntl_np, compression = 9)
        #bntl_np_tf = tf.io.serialize_tensor(bntl_np)

        ## Create dictionary
        feature = {
            'uid' : _bytes_feature(uid_i),
            'viirs_ntl_group' : _int64_feature(viirs_ntl_group),
            'year' : _int64_feature(survey_year_i),
            'b_rgb': _bytes_feature(brgb_np_tf),
            #'b_nir': _bytes_feature(bnir_np_tf)
            'b_ndvi': _bytes_feature(bndvi_np_tf),
            'b_bu': _bytes_feature(bbu_np_tf)
            }

        # Other MS Bands
        #b_other_list = []
        #for b_other_i in other_bs:
        #    bi_np = np.array(d_f_i[b_other_i])
        #    bi_np = np.expand_dims(bi_np, axis=2)
        #    #bi_np_tf = tf.io.serialize_tensor(bi_np)
        #    bi_np = bi_np.astype(np.uint16)
        #    bi_np_tf = tf.io.encode_png(bi_np, compression = 9)
        #    feature['b_' + b_other_i] = _bytes_feature(bi_np_tf)
  
        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))

        example_proto_list.append(example_proto)

        #out_file_name = os.path.join(out_path, folder_name, survey_uid + '.tfrecord')
        #with tf.io.TFRecordWriter(out_file_name) as writer:
        #  writer.write(example_proto.SerializeToString())
        
    return example_proto_list

        #bndvi_np = np.expand_dims(bndvi_l, axis=2)
        #b_np = np.expand_dims(b_l, axis=2)
        #b_np = np.repeat(b_np, 3, -1)
        #np.save(os.path.join(out_path, band_i + "_" + survey_uid + '.npy'), b_np)
        #np.save(os.path.join(out_path, 'BRGB' + "_" + survey_uid + '.npy'), brgb_np)
        #bndvi_np = np.repeat(bndvi_np, 3, -1)
        #np.save(os.path.join(out_path, 'BNDVI' + "_" + survey_uid + '.npy'), bndvi_np)

        #for band_i in SINGLE_BANDS_ALL:
        #    
        #    b_l = np.array(f_i[band_i])
        #    b_np = np.expand_dims(b_l, axis=2)
        #    #b_np = np.repeat(b_np, 3, -1)
        #    np.save(os.path.join(out_path, band_i + "_" + survey_uid + '.npy'), b_np)

    
    #return "Done"

def prep_cnn_np(survey_df,
                satellite_name,
                kernel_size,
                year):
    '''
    Creates numpy arrays for CNN

    Input:  df - pandas dataframe
            lat_name - name of latitude variable in df
            lon_name - name of longitude variable in df
    Output: geopandas dataframe
    '''

    # Setup --------------------------------------------------------------------
    # Survey to FeatureCollection  
    survey_fc = survey_to_fc(survey_df)

    # Define kernel for neighborhood array
    list = ee.List.repeat(1, kernel_size)
    lists = ee.List.repeat(list, kernel_size)
    kernel = ee.Kernel.fixed(kernel_size, kernel_size, lists)
    
    # Define satellite
    if satellite_name == 's2':
        satellite = 's2'
    elif satellite_name == 'landsat':
        if year >= 2014:
            satellite = 'l8'
        else:
            satellite = 'l7'
            
    # Define scale
    if satellite in ['l7', 'l8']:
        SCALE = 30
    elif satellite in ['s2']: 
        SCALE = 10

    # Prep NTL -----------------------------------------------------------------
    
    # Year
    # VIIRS starts in 2012. At minimum, use 2013 to have year before and after
    #if False:
    #    if year <= 2013:
    #        year_use = 2013
    #    else:
    #        year_use = year

    #    year_plus = year_use + 1
    #    year_minus = year_use - 1

    #    year_minus_str = str(year_minus) + '-01-01'
    #    year_plus_str = str(year_plus) + '-12-31'

        # Reduce image collection
    #    ntl_image = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')\
    #        .filterDate(year_minus_str, year_plus_str)\
    #        .median()

        # Select Bands  
    #    ntl_image = ntl_image.select(['avg_rad'])

        # Image to neighborhood array
    #    ntl_arrays = ntl_image.neighborhoodToArray(kernel)

        # Extract values from GEE    
    #    ntl_values_ee = ntl_arrays.sample(
    #      region = survey_fc, 
    #      scale = SCALE,
    #      tileScale = 10 #8
    #    )

    #    ntl_dict_ee = ntl_values_ee.getInfo()

        # Convert values to numpy array
        #n_rows = survey_df.shape[0]
    #    ntl_f = ntl_dict_ee['features']    
        
    # l7 ----------------------------------------------------------------
    if satellite == "l7":
        
        # Bands
        b_b = 'B1'
        g_b = 'B2' 
        r_b = 'B3' 
        nir_b = 'B4'
        swir_b = 'B5'
        #other_bs = ['B5', 'B6', 'B7']
        
        #BANDS = single_bs.copy()
        BANDS = [b_b].copy()
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        BANDS.append(swir_b)
        
        # Year
        # landsat 8 starts in May 1999; if year is less than
        # 2000, use 2000 as year (to ensure have year before and after)
        if year < 2000:
            year_use = 2000
        else:
            year_use = year

        # Year
        year_use = year
        
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median() #\
            #.multiply(0.0001)
    
    # l8 ----------------------------------------------------------------
    if satellite == "l8":
                
        # Bands
        # FOR COLLECTION 2
        #b_b = 'SR_B2'
        #g_b = 'SR_B3' 
        #r_b = 'SR_B4' 
        #nir_b = 'SR_B5'
        #other_bs = ['SR_B6', 'SR_B7', 'ST_B10']
        
        # FOR COLLECTION 1
        b_b = 'B2'
        g_b = 'B3' 
        r_b = 'B4' 
        nir_b = 'B5'
        swir_b = 'B6'
        #other_bs = ['B6', 'B7', 'B10']
        
        #BANDS = single_bs.copy()
        BANDS = [b_b].copy()
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        BANDS.append(swir_b)

        # Year
        # landsat 8 starts in April 2013; if year is less than
        # 2014, use 2014 as year (to ensure have year before and after)
        if year < 2014:
            year_use = 2014
        else:
            year_use = year
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        #image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        #    .filterDate(year_minus_str, year_plus_str)\
        #    #.map(cloud_mask_landsatSR)\ #TODO cloud_mask_landsatSR doesn't work with landsat collection 2
        #    .median() #\
        #    #.multiply(0.0001)
        
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median()
        
        #image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        #    .filterDate(year_minus_str, year_plus_str)\
        #    .median()
            
    # s2 ----------------------------------------------------------------
    if satellite == "s2":
        
        # Bands
        b_b = 'B2'
        g_b = 'B3' 
        r_b = 'B4' 
        nir_b = 'B8'
        swir_b = 'B11'
        #other_bs = ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'AOT']
     
        #BANDS = single_bs.copy()
        BANDS = [b_b].copy()
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        BANDS.append(swir_b)
        
        # Year
        # sentinel starts in March 2017; juse use 2018
        year_use = 2019
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'

        # Number of bands changes in sentinel, so need to select here before aggregate
        # https://gis.stackexchange.com/questions/374010/gee-tile-error-expected-a-homogeneous-image-collection-but-an-image-with-incom
        image = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_sentinel2)\
            .select(BANDS)\
            .median() # \
            #.multiply(0.0001)

    # Select Bands
    image = image.select(BANDS)
    
    # Create Indices
    # https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri
    ndvi = image.normalizedDifference([nir_b, r_b]).rename('NDVI');
    ndbi = image.normalizedDifference([swir_b, nir_b]).rename('NDBI');
    image = image.addBands(ndvi)
    image = image.addBands(ndbi)
        
    bu = image.select('NDBI').subtract(image.select('NDVI')).rename('BU')
    image = image.addBands(bu)
        
    # Subset bands; don't need those used to create NDVI and NDBI
    image = image.select([b_b, g_b, r_b, 'NDVI', 'BU'])
        
    # Image to neighborhood array
    arrays = image.neighborhoodToArray(kernel)
    
    # New ---------
    #neighborhoodImage = myImageToBeSampled.neighborhoodToArray(kernel)
    #samples = arrays.sampleRegions(collection=survey_fc)
    
    # ee.batch.Export.table.toCloudStorage
    # ee.batch.Export.table.toDrive
    # Export.table.toDrive
    #mytask = ee.batch.Export.table.toDrive(
    # collection = samples,
    # fileFormat = 'TFRecord',
    # description = 'test123',
    # folder = 'gee_extracts',
    # selectors = [b_b, g_b, r_b, 'NDVI', 'BU'] + ['uid', 'ntl_group'])
    
    #return mytask

    # OLD ---------
    # Extract values from GEE   
    values_ee = arrays.sample(
      region = survey_fc, 
      scale = SCALE,
      tileScale = 12 # 8
    )
    
    dict_ee = values_ee.getInfo()
     
    # Convert values to numpy array
    n_rows = survey_df.shape[0]
    daytime_f = dict_ee['features']
    
    # Extract data
    out_ex_proto_list = ee_to_np_daytime(daytime_f, survey_df, n_rows, b_b, g_b, r_b)
    
    return out_ex_proto_list

# https://csaybar.github.io/blog/2019/05/30/eetf/
# https://stackoverflow.com/questions/63000565/extract-10000-images-from-google-earth-engine
# https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb#scrollTo=-IlgXu-vcUEY

In [12]:
# Setup --------------------------------------------------------------------
# Survey to FeatureCollection  
survey_fc = survey_to_fc(survey_df)

# Define kernel for neighborhood array
list = ee.List.repeat(1, kernel_size)
lists = ee.List.repeat(list, kernel_size)
kernel = ee.Kernel.fixed(kernel_size, kernel_size, lists)

# Define satellite
if satellite_name == 's2':
    satellite = 's2'
elif satellite_name == 'landsat':
    if year >= 2014:
        satellite = 'l8'
    else:
        satellite = 'l7'

# Define scale
if satellite in ['l7', 'l8']:
    SCALE = 30
elif satellite in ['s2']: 
    SCALE = 10
    
# l7 ----------------------------------------------------------------
if satellite == "l7":

    # Bands
    b_b = 'B1'
    g_b = 'B2' 
    r_b = 'B3' 
    nir_b = 'B4'
    swir_b = 'B5'
    #other_bs = ['B5', 'B6', 'B7']

    #BANDS = single_bs.copy()
    BANDS = [b_b].copy()
    BANDS.append(g_b)
    BANDS.append(r_b)
    BANDS.append(nir_b)
    BANDS.append(swir_b)

    # Year
    # landsat 8 starts in May 1999; if year is less than
    # 2000, use 2000 as year (to ensure have year before and after)
    if year < 2000:
        year_use = 2000
    else:
        year_use = year

    # Year
    year_use = year

    year_plus = year_use + 1
    year_minus = year_use - 1

    year_minus_str = str(year_minus) + '-01-01'
    year_plus_str = str(year_plus) + '-12-31'

    image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
        .filterDate(year_minus_str, year_plus_str)\
        .map(cloud_mask_landsatSR)\
        .median() #\
        #.multiply(0.0001)

# l8 ----------------------------------------------------------------
if satellite == "l8":

    # Bands
    # FOR COLLECTION 2
    #b_b = 'SR_B2'
    #g_b = 'SR_B3' 
    #r_b = 'SR_B4' 
    #nir_b = 'SR_B5'
    #other_bs = ['SR_B6', 'SR_B7', 'ST_B10']

    # FOR COLLECTION 1
    b_b = 'B2'
    g_b = 'B3' 
    r_b = 'B4' 
    nir_b = 'B5'
    swir_b = 'B6'
    #other_bs = ['B6', 'B7', 'B10']

    #BANDS = single_bs.copy()
    BANDS = [b_b].copy()
    BANDS.append(g_b)
    BANDS.append(r_b)
    BANDS.append(nir_b)
    BANDS.append(swir_b)

    # Year
    # landsat 8 starts in April 2013; if year is less than
    # 2014, use 2014 as year (to ensure have year before and after)
    if year < 2014:
        year_use = 2014
    else:
        year_use = year

    year_plus = year_use + 1
    year_minus = year_use - 1

    year_minus_str = str(year_minus) + '-01-01'
    year_plus_str = str(year_plus) + '-12-31'

    #image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    #    .filterDate(year_minus_str, year_plus_str)\
    #    #.map(cloud_mask_landsatSR)\ #TODO cloud_mask_landsatSR doesn't work with landsat collection 2
    #    .median() #\
    #    #.multiply(0.0001)

    image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
        .filterDate(year_minus_str, year_plus_str)\
        .map(cloud_mask_landsatSR)\
        .median()

    #image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    #    .filterDate(year_minus_str, year_plus_str)\
    #    .median()

# s2 ----------------------------------------------------------------
if satellite == "s2":

    # Bands
    b_b = 'B2'
    g_b = 'B3' 
    r_b = 'B4' 
    nir_b = 'B8'
    swir_b = 'B11'
    #other_bs = ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'AOT']

    #BANDS = single_bs.copy()
    BANDS = [b_b].copy()
    BANDS.append(g_b)
    BANDS.append(r_b)
    BANDS.append(nir_b)
    BANDS.append(swir_b)

    # Year
    # sentinel starts in March 2017; juse use 2018
    year_use = 2019

    year_plus = year_use + 1
    year_minus = year_use - 1

    year_minus_str = str(year_minus) + '-01-01'
    year_plus_str = str(year_plus) + '-12-31'

    # Number of bands changes in sentinel, so need to select here before aggregate
    # https://gis.stackexchange.com/questions/374010/gee-tile-error-expected-a-homogeneous-image-collection-but-an-image-with-incom
    image = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterDate(year_minus_str, year_plus_str)\
        .map(cloud_mask_sentinel2)\
        .select(BANDS)\
        .median() # \
        #.multiply(0.0001)

# Select Bands
image = image.select(BANDS)

# Create Indices
# https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri
ndvi = image.normalizedDifference([nir_b, r_b]).rename('NDVI');
ndbi = image.normalizedDifference([swir_b, nir_b]).rename('NDBI');
image = image.addBands(ndvi)
image = image.addBands(ndbi)

bu = image.select('NDBI').subtract(image.select('NDVI')).rename('BU')
image = image.addBands(bu)

# Subset bands; don't need those used to create NDVI and NDBI
image = image.select([b_b, g_b, r_b, 'NDVI', 'BU'])

# Image to neighborhood array
arrays = image.neighborhoodToArray(kernel)

# New ---------
#neighborhoodImage = myImageToBeSampled.neighborhoodToArray(kernel)
#samples = arrays.sampleRegions(collection=survey_fc)

# ee.batch.Export.table.toCloudStorage
# ee.batch.Export.table.toDrive
# Export.table.toDrive
#mytask = ee.batch.Export.table.toDrive(
# collection = samples,
# fileFormat = 'TFRecord',
# description = 'test123',
# folder = 'gee_extracts',
# selectors = [b_b, g_b, r_b, 'NDVI', 'BU'] + ['uid', 'ntl_group'])

#return mytask

# OLD ---------
# Extract values from GEE   
values_ee = arrays.sample(
  region = survey_fc, 
  scale = SCALE,
  tileScale = 12 # 8
)

dict_ee = values_ee.getInfo()

# Convert values to numpy array
n_rows = survey_df.shape[0]
daytime_f = dict_ee['features']

# Extract data
out_ex_proto_list = ee_to_np_daytime(daytime_f, survey_df, n_rows, b_b, g_b, r_b)


AttributeError: module 'tensorflow_core._api.v2.io' has no attribute 'encode_png'

In [ ]:
def prep_cnn_np(survey_df,
                satellite_name,
                kernel_size,
                year):
    '''
    Creates numpy arrays for CNN

    Input:  df - pandas dataframe
            lat_name - name of latitude variable in df
            lon_name - name of longitude variable in df
    Output: geopandas dataframe
    '''

    # Setup --------------------------------------------------------------------
    # Survey to FeatureCollection  
    survey_fc = survey_to_fc(survey_df)

    # Define kernel for neighborhood array
    list = ee.List.repeat(1, kernel_size)
    lists = ee.List.repeat(list, kernel_size)
    kernel = ee.Kernel.fixed(kernel_size, kernel_size, lists)
    
    # Define satellite
    if satellite_name == 's2':
        satellite = 's2'
    elif satellite_name == 'landsat':
        if year >= 2014:
            satellite = 'l8'
        else:
            satellite = 'l7'
            
    # Define scale
    if satellite in ['l7', 'l8']:
        SCALE = 30
    elif satellite in ['s2']: 
        SCALE = 10

    # Prep NTL -----------------------------------------------------------------
    
    # Year
    # VIIRS starts in 2012. At minimum, use 2013 to have year before and after
    #if False:
    #    if year <= 2013:
    #        year_use = 2013
    #    else:
    #        year_use = year

    #    year_plus = year_use + 1
    #    year_minus = year_use - 1

    #    year_minus_str = str(year_minus) + '-01-01'
    #    year_plus_str = str(year_plus) + '-12-31'

        # Reduce image collection
    #    ntl_image = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')\
    #        .filterDate(year_minus_str, year_plus_str)\
    #        .median()

        # Select Bands  
    #    ntl_image = ntl_image.select(['avg_rad'])

        # Image to neighborhood array
    #    ntl_arrays = ntl_image.neighborhoodToArray(kernel)

        # Extract values from GEE    
    #    ntl_values_ee = ntl_arrays.sample(
    #      region = survey_fc, 
    #      scale = SCALE,
    #      tileScale = 10 #8
    #    )

    #    ntl_dict_ee = ntl_values_ee.getInfo()

        # Convert values to numpy array
        #n_rows = survey_df.shape[0]
    #    ntl_f = ntl_dict_ee['features']    
        
    # l7 ----------------------------------------------------------------
    if satellite == "l7":
        
        # Bands
        b_b = 'B1'
        g_b = 'B2' 
        r_b = 'B3' 
        nir_b = 'B4'
        swir_b = 'B5'
        #other_bs = ['B5', 'B6', 'B7']
        
        #BANDS = single_bs.copy()
        BANDS = [b_b].copy()
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        BANDS.append(swir_b)
        
        # Year
        # landsat 8 starts in May 1999; if year is less than
        # 2000, use 2000 as year (to ensure have year before and after)
        if year < 2000:
            year_use = 2000
        else:
            year_use = year

        # Year
        year_use = year
        
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('LANDSAT/LC07/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median() #\
            #.multiply(0.0001)
    
    # l8 ----------------------------------------------------------------
    if satellite == "l8":
                
        # Bands
        # FOR COLLECTION 2
        #b_b = 'SR_B2'
        #g_b = 'SR_B3' 
        #r_b = 'SR_B4' 
        #nir_b = 'SR_B5'
        #other_bs = ['SR_B6', 'SR_B7', 'ST_B10']
        
        # FOR COLLECTION 1
        b_b = 'B2'
        g_b = 'B3' 
        r_b = 'B4' 
        nir_b = 'B5'
        swir_b = 'B6'
        #other_bs = ['B6', 'B7', 'B10']
        
        #BANDS = single_bs.copy()
        BANDS = [b_b].copy()
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        BANDS.append(swir_b)

        # Year
        # landsat 8 starts in April 2013; if year is less than
        # 2014, use 2014 as year (to ensure have year before and after)
        if year < 2014:
            year_use = 2014
        else:
            year_use = year
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        #image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        #    .filterDate(year_minus_str, year_plus_str)\
        #    #.map(cloud_mask_landsatSR)\ #TODO cloud_mask_landsatSR doesn't work with landsat collection 2
        #    .median() #\
        #    #.multiply(0.0001)
        
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median()
        
        #image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        #    .filterDate(year_minus_str, year_plus_str)\
        #    .median()
            
    # s2 ----------------------------------------------------------------
    if satellite == "s2":
        
        # Bands
        b_b = 'B2'
        g_b = 'B3' 
        r_b = 'B4' 
        nir_b = 'B8'
        swir_b = 'B11'
        #other_bs = ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'AOT']
     
        #BANDS = single_bs.copy()
        BANDS = [b_b].copy()
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        BANDS.append(swir_b)
        
        # Year
        # sentinel starts in March 2017; juse use 2018
        year_use = 2019
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'

        # Number of bands changes in sentinel, so need to select here before aggregate
        # https://gis.stackexchange.com/questions/374010/gee-tile-error-expected-a-homogeneous-image-collection-but-an-image-with-incom
        image = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_sentinel2)\
            .select(BANDS)\
            .median() # \
            #.multiply(0.0001)

    # Select Bands
    image = image.select(BANDS)
    
    # Create Indices
    # https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri
    ndvi = image.normalizedDifference([nir_b, r_b]).rename('NDVI');
    ndbi = image.normalizedDifference([swir_b, nir_b]).rename('NDBI');
    image = image.addBands(ndvi)
    image = image.addBands(ndbi)
        
    bu = image.select('NDBI').subtract(image.select('NDVI')).rename('BU')
    image = image.addBands(bu)
        
    # Subset bands; don't need those used to create NDVI and NDBI
    image = image.select([b_b, g_b, r_b, 'NDVI', 'BU'])
        
    # Image to neighborhood array
    arrays = image.neighborhoodToArray(kernel)
    
    # New ---------
    #neighborhoodImage = myImageToBeSampled.neighborhoodToArray(kernel)
    #samples = arrays.sampleRegions(collection=survey_fc)
    
    # ee.batch.Export.table.toCloudStorage
    # ee.batch.Export.table.toDrive
    # Export.table.toDrive
    #mytask = ee.batch.Export.table.toDrive(
    # collection = samples,
    # fileFormat = 'TFRecord',
    # description = 'test123',
    # folder = 'gee_extracts',
    # selectors = [b_b, g_b, r_b, 'NDVI', 'BU'] + ['uid', 'ntl_group'])
    
    #return mytask

    # OLD ---------
    # Extract values from GEE   
    values_ee = arrays.sample(
      region = survey_fc, 
      scale = SCALE,
      tileScale = 12 # 8
    )
    
    dict_ee = values_ee.getInfo()
     
    # Convert values to numpy array
    n_rows = survey_df.shape[0]
    daytime_f = dict_ee['features']
    
    # Extract data
    out_ex_proto_list = ee_to_np_daytime(daytime_f, survey_df, n_rows, b_b, g_b, r_b)
    
    return out_ex_proto_list


In [13]:
bndvi_np_tf = tf.io.encode_png(bndvi_np, compression = 9)

AttributeError: module 'tensorflow' has no attribute 'encode_png'

In [14]:
tf.io.

NameError: name 'bndvi_np' is not defined

In [9]:
# DONT SKIP ERRORS
if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.

        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            #time.sleep(6)
            
            ## Sometimes we hit a memory error; try until we don't hit that
            
            # TODO: could say: try 3 times?
            try_extract_data = True
            while try_extract_data:
                try:

                    print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                    survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                    proto_examples_all.extend(proto_examples_i)
                    
                    try_extract_data = False
                    
                except:
                    print("Error!")
                    time.sleep(10)
                    pass
            
        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Putting 66 observations into forcnn_AL_1_1_all.tfrecord
Observation: 0/66
Observation: 1/66
Observation: 2/66
Observation: 3/66
Error!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-9-35e4ef9beeda>", line 32, in <module>
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
  File "/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py", line 443, in prep_cnn_np
    dict_ee = values_ee.getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/collection.py", line 127, in getInfo
    return super(Collection, self).getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/computedobject.py", line 98, in getInfo


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-9-35e4ef9beeda>", line 32, in <module>
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
  File "/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py", line 443, in prep_cnn_np
    dict_ee = values_ee.getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/collection.py", line 127, in getInfo
    return super(Collection, self).getInfo()
  File "/Users/robmarty/anaconda3/lib/python3.7/site-packages/ee/computedobject.py", line 98, in getInfo


TypeError: can only concatenate str (not "list") to str

In [ ]:
# SKIP ERRORS

### Loop through all tfrecords
for tfr_i in tf_record_list:

    # Sometimes we get computational time out errors. If occurs, just skip and go to next.
    # We can then go back and rescrape missed ones.

    survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
    year_i = survey_df_yeari['year'].iloc[0]

    ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
    survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

    print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

    proto_examples_all = []
    for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
        ## Sometimes we hit a memory error; try until we don't hit that

        # TODO: could say: try 3 times?
        try_extract_data = 1
        while try_extract_data < 4:
            try:

                print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                proto_examples_all.extend(proto_examples_i)
                
                try_extract_data = 10

            except:
                try_extract_data = try_extract_data + 1
                print("Error!")
                print(try_extract_data)
                print(survey_df_yeari_chunki['uid'])
                time.sleep(5)
                pass

    ### Save data as tf record
    out_path_i = os.path.join(out_path, tfr_i)
    print(out_path_i)
    with tf.io.TFRecordWriter(out_path_i) as writer:
        for tf_example in proto_examples_all:
            writer.write(tf_example.SerializeToString())

    print("Success \o/")

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Putting 36 observations into forcnn_UG_4_1_all.tfrecord
Observation: 0/36
Observation: 1/36
Observation: 2/36
Observation: 3/36
Observation: 4/36
Observation: 5/36
Error ---
2161    UG201800000328
Name: uid, dtype: object
Putting 246 observations into nocnn_IA_4_10_all.tfrecord
Observation: 0/246
Observation: 1/246
Observation: 2/246
Observation: 3/246
Observation: 4/246
Observation: 5/246
Observation: 6/246
Observation: 7/246
Observation: 8/246
Observation: 9/246
Observation: 10/246
Observation: 11/246
Observation: 12/246
Observation: 13/246
Observation: 14/246
Observation: 15/246
Observation: 16/246
Observation: 17/246
Observation: 18/246
Observation: 19/246
Observation: 20/246
Observation: 21/246
Observation: 22/246
Observation: 23/246
Observation: 24/246
Observation: 25/246
Observation: 26/246
Observation: 27/246
Observation: 28/246
Observation: 29/246
Observation: 30/246
Observation: 31/246
Observation: 32/246
Observation: 33/246
Observation: 34/246
Observation: 35/246
Observation

In [10]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

0
1
2
3
4


IndexError: positional indexers are out-of-bounds

In [ ]:
survey_df_yeari_chunki.iloc[[4]]